Hate Speech Detection in Social Media Using Advanced Machine Learning Algorithms

In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import tensorflow_hub as hub
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report
import warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

In [4]:
import nltk
import re
from nltk.corpus import stopwords
stopword = set(stopwords.words('english'))
stemmer = nltk.SnowballStemmer("english")

In [5]:
data = pd.read_csv("labeled_data.csv")
print(data.head())

   Unnamed: 0  count  hate_speech  offensive_language  neither  class  \
0           0      3            0                   0        3      2   
1           1      3            0                   3        0      1   
2           2      3            0                   3        0      1   
3           3      3            0                   2        1      1   
4           4      6            0                   6        0      1   

                                               tweet  
0  !!! RT @mayasolovely: As a woman you shouldn't...  
1  !!!!! RT @mleew17: boy dats cold...tyga dwn ba...  
2  !!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...  
3  !!!!!!!!! RT @C_G_Anderson: @viva_based she lo...  
4  !!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...  


In [6]:
data["labels"] = data["class"].map({0:"Hate Speech",1:"Offensive Speech",2:"No Hate and Offensive Speech"})

In [7]:
data = data[["tweet","labels"]]

In [8]:
data.head()

,tweet,labels
0,!!! RT @mayasolovely: As a woman you shouldn't...,No Hate and Offensive Speech
1,!!!!! RT @mleew17: boy dats cold...tyga dwn ba...,Offensive Speech
2,!!!!!!! RT @UrKindOfBrand Dawg!!!! RT @80sbaby...,Offensive Speech
3,!!!!!!!!! RT @C_G_Anderson: @viva_based she lo...,Offensive Speech
4,!!!!!!!!!!!!! RT @ShenikaRoberts: The shit you...,Offensive Speech


In [9]:
def clean(text):
    text = str(text).lower()
    text = re.sub('[.?]', '', text)
    text = re.sub('https?://\S+|www.\S+', '', text)
    text = re.sub('<.?>+', '', text)
    text = re.sub(r'[^\w\s]', '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w\d\w', '', text)
    text = [word for word in text.split(' ') if word not in stopword]
    text = " ".join(text)
    text =[stemmer.stem(word) for word in text.split(' ')]
    text = " ".join(text)
    return text
    data["tweet"] = data["tweet"].apply(clean)

In [10]:
x = np.array(data["tweet"])
y = np.array(data["labels"])

In [11]:
cv = CountVectorizer()

In [12]:
X = cv.fit_transform(x)

In [13]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)

In [14]:
model = DecisionTreeClassifier()

In [15]:
model.fit(X_train, y_train)

DecisionTreeClassifier()

In [16]:
y_pred = model.predict(X_test)

In [17]:
from sklearn.metrics import accuracy_score
print(accuracy_score(y_test, y_pred)*100)

88.92285120430371


In [18]:
i =" got beaten by a thug"
i = cv.transform([i]).toarray()
print(model.predict(i))

['No Hate and Offensive Speech']


In [19]:
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(max_iter = 100)
logreg.fit(X_train, y_train)
logreg_predict = logreg.predict(X_test)
logreg_acc = accuracy_score(logreg_predict, y_test)

In [20]:
print("Test accuracy: {:.2f}".format(logreg_acc*100))

Test accuracy: 89.96


In [21]:
print(classification_report(y_test, logreg_predict))

                              precision    recall  f1-score   support

                 Hate Speech       0.48      0.25      0.33       465
No Hate and Offensive Speech       0.84      0.87      0.85      1379
            Offensive Speech       0.93      0.95      0.94      6335

                    accuracy                           0.90      8179
                   macro avg       0.75      0.69      0.71      8179
                weighted avg       0.89      0.90      0.89      8179



In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

X_train, X_test, y_train, y_test = train_test_split(data['tweet'], data['labels'], test_size= 0.2, random_state=42)

tfidf_vectorizer = TfidfVectorizer(max_features=5000) 
X_train_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_tfidf = tfidf_vectorizer.transform(X_test)

rf_classifier = RandomForestClassifier(n_estimators=100, random_state=42)
rf_classifier.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = rf_classifier.predict(X_test_tfidf)
accuracy = accuracy_score(y_test, y_pred)
print("Test accuracy: {:.2f}%".format(accuracy*100))

In [ ]:
from sklearn.naive_bayes import MultinomialNB

In [ ]:
naive_bayes_classifier = MultinomialNB()
naive_bayes_classifier.fit(X_train_tfidf, y_train)

In [ ]:
y_pred = naive_bayes_classifier.predict(X_test_tfidf)

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)
print('Accuracy:', accuracy*100)
print('Classification Report:\n', report)

In [ ]:
def build_CNN_classifier_model():
    text_input = tf.keras.layers.Input(shape=(), dtype=tf.string, name='text')
    preprocessing_layer = hub.KerasLayer(tfhub_handle_preprocess, name='preprocessing')
    encoder_inputs = preprocessing_layer(text_input)
    encoder = hub.KerasLayer(tfhub_handle_encoder, trainable=True, name='BERT_encoder')
    outputs = encoder(encoder_inputs)
    #net = outputs['pooled_output'] # [batch_size, 768].
    net = sequence_output = outputs["sequence_output"] # [batch_size, seq_length, 768]


    net = tf.keras.layers.Conv1D(32, (2), activation='relu')(net)
    #net = tf.keras.layers.MaxPooling1D(2)(net)

    net = tf.keras.layers.Conv1D(64, (2), activation='relu')(net)
    #net = tf.keras.layers.MaxPooling1D(2)(net)
    net = tf.keras.layers.GlobalMaxPool1D()(net)

#    net = tf.keras.layers.Flatten()(net)

    net = tf.keras.layers.Dense(768, activation="relu")(net)

    net = tf.keras.layers.Dropout(0.1)(net)
#   net = tf.keras.layers.Dense(1, activation=None, name='classifier')(net)
    net = tf.keras.layers.Dense(3, activation="softmax", name='classifier')(net)

    return tf.keras.Model(text_input, net)

In [ ]:
cnn_classifier_model = build_CNN_classifier_model()
bert_raw_result = cnn_classifier_model(tf.constant(text_test))
print(tf.sigmoid(bert_raw_result))

In [ ]:
text_test

In [ ]:
cnn_classifier_model.summary()

In [ ]:
tf.keras.utils.plot_model(cnn_classifier_model)

In [ ]:
loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
#metrics = tf.metrics.CategoricalCrossentropy()
#metrics = tf.metrics.Accuracy()

In [ ]:
epochs = 80
steps_per_epoch = tf.data.experimental.cardinality(train_ds).numpy()
num_train_steps = steps_per_epoch * epochs
num_warmup_steps = int(0.1*num_train_steps)

init_lr = 3e-5
optimizer = optimization.create_optimizer(init_lr=init_lr,
                                          num_train_steps=num_train_steps,
                                          num_warmup_steps=num_warmup_steps,
                                          optimizer_type='adamw')

cnn_classifier_model.compile(optimizer=optimizer,
                          loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
                          metrics=tf.keras.metrics.SparseCategoricalAccuracy('accuracy'))

In [ ]:
print(f'Training model with {tfhub_handle_encoder}')
cnn_history = cnn_classifier_model.fit(x=train_ds,
                                       validation_data=val_ds,
                                       epochs=epochs,
                                       class_weight=class_weight
                                      )

In [ ]:
loss, accuracy = cnn_classifier_model.evaluate(test_ds)

print(f'Loss: {loss}')
print(f'Accuracy: {accuracy}')

In [ ]:
history_dict = cnn_history.history
print(history_dict.keys())

acc = history_dict['accuracy']
val_acc = history_dict['val_accuracy']
# acc = history_dict['binary_accuracy']
# val_acc = history_dict['val_binary_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)
fig = plt.figure(figsize=(12, 10))
fig.tight_layout()

plt.subplot(2, 1, 1)
# "bo" is for "blue dot"
plt.plot(epochs, loss, 'r', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
# plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.subplot(2, 1, 2)
plt.plot(epochs, acc, 'r', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

In [ ]:
dataset_name = 'cnn_hate_speech'
saved_model_path = './{}_bert'.format(dataset_name.replace('/', '_'))

cnn_classifier_model.save(saved_model_path, include_optimizer=False)

In [ ]:
reloaded_model = tf.saved_model.load(saved_model_path)

# ROBERTa

In [ ]:
!pip -q install evaluate peft
from transformers import pipeline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
from datasets import Dataset, DatasetDict
import evaluate
import os
import torch

pd.set_option('display.max_colwidth', None)
os.environ["WANDB_DISABLED"] = "true"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model_name = "facebook/roberta-hate-speech-dynabench-r4-target"
pipe = pipeline("text-classification", model=model_name, device=device)
dfRaw = pd.read_csv('/kaggle/input/hate-speech-and-offensive-language-dataset/labeled_data.csv')
dfRaw.head()
print('hate_speech value distribution', dfRaw.offensive_language.value_counts())
df = dfRaw[((dfRaw['hate_speech'] >= 2) & (dfRaw['class'] == 0)) | ((dfRaw['neither'] >= 2) & (dfRaw['class'] == 2))]
df.head()
df.insert(1, 'hate', 0)
df.loc[df['hate_speech'] >= 2, 'hate'] = 1
df = df[['hate', 'tweet']]
df.head()
df.info()
df.hate.value_counts()
tweets = df.iloc[:3]['tweet'].tolist()
outs = pipe(tweets)
# print(tweets)
print(outs)
print([out['label'] == 'hate' for out in outs])
n_samples = 1000
sampleDf = df.iloc[:n_samples]
preds = pipe(df.tweet.tolist()[:n_samples])
sampleDf.insert(1, 'predictions_hate', [int(pred['label'] == 'hate') for pred in preds])
# sampleDf.loc[:n_samples, 'predictions_hate'] = [int(pred['label'] == 'hate') for pred in preds]
# sampleDf.predictions_hate = sampleDf.predictions_hate.astype(np.int32)
sampleDf
accuracy = len(sampleDf[sampleDf['hate'] == sampleDf['predictions_hate']]) / len(sampleDf)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

cm = confusion_matrix(sampleDf['hate'], sampleDf['predictions_hate'], labels=[0, 1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['Not Hate', 'Hate'])
disp.plot()
plt.show()
dfTrain = df.rename(columns={
    'tweet': 'text',
    'hate': 'label'
})
dfTrain.head()
indexTrain = round(0.8*len(dfTrain))
indexVal = round(0.9*len(dfTrain))
indexTest = len(dfTrain)
train = dfTrain.iloc[:indexTrain]
val = dfTrain.iloc[indexTrain:indexVal]
test = dfTrain.iloc[indexVal:]
print(len(train), len(val), len(test))
dataset = DatasetDict()
dataset['train'] = Dataset.from_pandas(train)
dataset['val'] = Dataset.from_pandas(val)
dataset['test'] = Dataset.from_pandas(test)
dataset
sampleDf = test
preds = pipe(test.text.tolist()[:n_samples])
sampleDf.insert(1, 'predictions_hate', [int(pred['label'] == 'hate') for pred in preds])
# sampleDf.loc[:n_samples, 'predictions_hate'] = [int(pred['label'] == 'hate') for pred in preds]
# sampleDf.predictions_hate = sampleDf.predictions_hate.astype(np.int32)
sampleDf

accuracy = len(sampleDf[sampleDf['label'] == sampleDf['predictions_hate']]) / len(sampleDf)
print(f"Overall Accuracy: {accuracy * 100:.2f}%")

cm = confusion_matrix(sampleDf['label'], sampleDf['predictions_hate'], labels=[0, 1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['Not Hate', 'Hate'])
disp.plot()
plt.show()
from transformers import AutoTokenizer, DataCollatorWithPadding

tokenizer = AutoTokenizer.from_pretrained(model_name)


def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)


tokenized_datasets = dataset.map(tokenize_function, batched=True)
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(labels, preds, average="weighted")
    acc = accuracy_score(labels, preds)
    return {"accuracy": acc, "f1": f1}
id2label = {0: "hate", 1: "not hate"}
label2id = {"hate": 0, "not hate": 1}
from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, id2label=id2label, label2id=label2id).to(device)
from transformers import TrainingArguments, Trainer
from peft import LoraConfig, get_peft_model, TaskType

lora_config = LoraConfig(
    r=32, # Rank
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type=TaskType.SEQ_CLS)
output_dir = f'/kaggle/working/peft-training'
batch_size = 1

peft_model = get_peft_model(model, 
                            lora_config)

peft_training_args = TrainingArguments(
    output_dir=output_dir,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    learning_rate=1e-3,
    max_steps = 1,
    logging_steps=1,
    load_best_model_at_end=True,
    evaluation_strategy='steps',
    save_strategy='steps'
)
    
peft_trainer = Trainer(
    model=peft_model,
    args=peft_training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["val"],
)
peft_trainer.train()
peft_model.eval()
pred = test.copy()
preds = []
for text in test['text']:
    inputs = tokenizer([text], return_tensors="pt", padding=True)
    logits = peft_model(input_ids=inputs["input_ids"].to(device)).logits
    hate = int(logits.softmax(dim=-1).tolist()[0][1] > 0.7)
    preds.append(hate)
    
pred.insert(1, 'pred', preds)
pred.head()
pred['pred'].value_counts()
accuracy_after = len(pred[pred['label'] == pred['pred']]) / len(pred)
print(f"Overall Accuracy: {accuracy_after * 100:.2f}%")

cm = confusion_matrix(pred['label'], pred['pred'], labels=[0, 1])
disp = ConfusionMatrixDisplay(confusion_matrix=cm,
                              display_labels=['Not Hate', 'Hate'])
disp.plot()
plt.show()
improvement = accuracy_after - accuracy
print(f"Accuracy Improvement: {improvement * 100:.2f}%")

# BERT

In [1]:
!pip install transformers
!pip install datasets
from datasets import load_dataset, DatasetDict
from transformers import AutoTokenizer, TFAutoModelForSequenceClassification
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
data_path='/kaggle/input/hate-speech-and-offensive-language-dataset/labeled_data.csv'
df=pd.read_csv(data_path)
df.head()
import seaborn as sns
sns.countplot(x='class',data=df)
df['tweet_cleaned']=df['tweet'].str.replace('@[A-Za-z0-9]+\s?', '', regex=True)
df.head()
from datasets import Dataset
ds = Dataset.from_pandas(df)
ds
dataset = load_dataset('csv', data_files=data_path, split='train')
dataset
train_test_valid = ds.train_test_split()
test_valid = train_test_valid['test'].train_test_split()
train_test_valid_dataset = DatasetDict({
    'train': train_test_valid['train'],
    'test': test_valid['test'],
    'valid': test_valid['train']
 })

dataset = train_test_valid_dataset.remove_columns(['hate_speech','offensive_language','neither','Unnamed: 0','count'])
dataset
tokenizer = AutoTokenizer.from_pretrained('bert-base-cased')
text = "just checking tokenization"
output = tokenizer(text)
output
tokens = tokenizer.convert_ids_to_tokens(output['input_ids'])
tokens
print(f"Tokenized text: {tokenizer.convert_tokens_to_string(tokens)}")
print(f"Vocab_size is: {tokenizer.vocab_size}")

print(f"Model max length is: {tokenizer.model_max_length}")

print(f"Model input names are: {tokenizer.model_input_names}")
def tokenize_function(train_dataset):
  return tokenizer(train_dataset['tweet_cleaned'], padding='max_length', truncation=True)


tokenized_dataset = dataset.map(tokenize_function, batched=True)

train_dataset = tokenized_dataset['train']
eval_dataset = tokenized_dataset['valid']
test_dataset = tokenized_dataset['test']
train_dataset
train_set = train_dataset.remove_columns(['tweet',"tweet_cleaned"]).with_format('tensorflow')
tf_eval_dataset = eval_dataset.remove_columns(['tweet',"tweet_cleaned"]).with_format('tensorflow')
tf_test_dataset = test_dataset.remove_columns(['tweet',"tweet_cleaned"]).with_format('tensorflow')
train_features = {x: train_set[x] for x in tokenizer.model_input_names}
train_set_for_final_model = tf.data.Dataset.from_tensor_slices((train_features, train_set['class']))
train_set_for_final_model = train_set_for_final_model.shuffle(len(train_set)).batch(8)

eval_features = {x: tf_eval_dataset[x] for x in tokenizer.model_input_names}
val_set_for_final_model = tf.data.Dataset.from_tensor_slices((eval_features, tf_eval_dataset["class"]))
val_set_for_final_model = val_set_for_final_model.batch(8)

test_features = {x:tf_test_dataset[x] for x in tokenizer.model_input_names}
test_set_for_final_model = tf.data.Dataset.from_tensor_slices((test_features, tf_test_dataset["class"]))
test_set_for_final_model = test_set_for_final_model.batch(8)
model = TFAutoModelForSequenceClassification.from_pretrained('bert-base-cased', num_labels=3)
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=tf.keras.metrics.SparseCategoricalAccuracy(),
)
model.fit(train_set_for_final_model, validation_data=val_set_for_final_model, epochs=3)
history = model.fit(train_set_for_final_model, validation_data=val_set_for_final_model, epochs=3)
plt.plot(history.history['sparse_categorical_accuracy'])
plt.plot(history.history['val_sparse_categorical_accuracy'])
plt.title('model sparse categorical accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()

plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'val'], loc='upper left')
plt.show()
test_loss, test_acc = model.evaluate(test_set_for_final_model,verbose=2)
print('\nTest accuracy:', test_acc)
predict_score_and_class_dict = {
    0: 'Hate Speech',
    1: 'Offensive Language',
    2: 'Neither'}

preds = model(tokenizer(["He is useless, I dont know why he came to our neighbourhood", "That guy sucks", "He is such a retard"],
                        return_tensors="tf",padding=True,truncation=True))['logits']

print(preds)

class_preds = np.argmax(preds, axis=1)

for pred in class_preds:
  print(predict_score_and_class_dict[pred])



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip



[notice] A new release of pip is available: 24.0 -> 24.2
[notice] To update, run: python.exe -m pip install --upgrade pip
C:\Users\USER\anaconda3\envs\TABAMachineLearning\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


ModuleNotFoundError: No module named 'matplotlib'